# Processing the US EPA's 2017 National Emissions Inventory

This notebook contains a work-in-progress effort for processing the 2017 NEI using the [Julia language](https://julialang.org/).

First, we need to load the packages that we will use:

In [1]:
using Revise
using CSV
using ZipFile
using DataFrames
using HTTP
using Unitful

### Set download directory

Next, we need to choose a directory on our computer to download the emissions data to. You will probably need to change the directory below to a location that exists on your computer

In [ ]:
dir = "/Users/$(ENV["USER"])/data/2017nei"

## Download emissions files from the EPA web server

In this step, we download and unzip all of the emissions files.

You only need to do this once.

In [ ]:
urls = [
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017NEI_SmokeFlatFile_POINT_20200412_sectorized.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_CMV_12US1_29jun2020.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_cem_29jun2020.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_fires_29jun2020.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_nonpoint_29jun2020.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_nonroad_29jun2020.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_onroad_activity_29jun2020.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_oth_29jun2020.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_point_09feb2021.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_inventory_point_29jun2020.zip",
    "https://gaftp.epa.gov/Air/emismod/2017/2017emissions/2017gb_onroad_SMOKE-MOVES_emissions_FF10_29jun2020.zip",
]

for url in urls
    println("Downloading $(url)")
    http_response = HTTP.get(url, require_ssl_verification = false)
    r = ZipFile.Reader(IOBuffer(http_response.body));
    for f in r.files
        println("Filename: $(f.name)")
        if f.name[end] == '/'
            mkpath(joinpath(dir, f.name))
            continue
        end
        o = open(joinpath(dir, f.name), "w")
        write(o, f);
    end
end

### Utility functions

This next cell contains some code that we will use to load and process the emissions. It will eventually be moved into a separate software library.

In [17]:
const tonperyear = 907.185u"kg" / 31_536_000u"s"
const tonpermonth = 907.185u"kg" / 2_628_288u"s"
const foot = (1/3.28084)u"m"
kelvin(F) = ((F − 32.0) * 5.0/9.0 + 273.15) * u"K"

abstract type EmissionsDataFrame end

# https://www.cmascenter.org/smoke/documentation/4.8.1/html/ch08s02s04.html#d0e38214
struct FF10NonPointDataFrame <: EmissionsDataFrame
    df::DataFrame
    
    FF10NonPointDataFrame(df::DataFrame) = begin
        if size(df, 2) != 45
            throw(DimensionMismatch("FF10 nonpoint file should have 45 fields but instead has $(size(df,2))"))
        end
        
        rename!(df, ["COUNTRY", "FIPS", "TRIBAL_CODE", "CENSUS_TRACT", "SHAPE_ID", "SCC",
            "EMIS_TYPE", "POLID", "ANN_VALUE",
            "ANN_PCT_RED", "CONTROL_IDS", "CONTROL_MEASURES", "CURRENT_COST", "CUMULATIVE_COST", "PROJECTION_FACTOR",
            "REG_CODES", "CALC_METHOD", "CALC_YEAR", "DATE_UPDATED", "DATA_SET_ID", "JAN_VALUE", "FEB_VALUE", "MAR_VALUE",
            "APR_VALUE", "MAY_VALUE", "JUN_VALUE", "JUL_VALUE", "AUG_VALUE", "SEP_VALUE", "OCT_VALUE", "NOV_VALUE", "DEC_VALUE",
            "JAN_PCTRED", "FEB_PCTRED", "MAR_PCTRED", "APR_PCTRED", "MAY_PCTRED", "JUN_PCTRED", "JUL_PCTRED", "AUG_PCTRED",
            "SEP_PCTRED", "OCT_PCTRED", "NOV_PCTRED", "DEC_PCTRED", "COMMENT"])
        
        df.ANN_VALUE = df.ANN_VALUE * tonperyear
        
        df.JAN_VALUE = df.JAN_VALUE * tonpermonth
        df.FEB_VALUE = df.FEB_VALUE * tonpermonth
        df.MAR_VALUE = df.MAR_VALUE * tonpermonth
        df.APR_VALUE = df.APR_VALUE * tonpermonth
        df.MAY_VALUE = df.MAY_VALUE * tonpermonth
        df.JUN_VALUE = df.JUN_VALUE * tonpermonth
        df.JUL_VALUE = df.JUL_VALUE * tonpermonth
        df.AUG_VALUE = df.AUG_VALUE * tonpermonth
        df.SEP_VALUE = df.SEP_VALUE * tonpermonth
        df.OCT_VALUE = df.OCT_VALUE * tonpermonth
        df.NOV_VALUE = df.NOV_VALUE * tonpermonth
        df.DEC_VALUE = df.DEC_VALUE * tonpermonth
        
        return new(df)
    end
end

struct FF10NonRoadDataFrame <: EmissionsDataFrame
    df::DataFrame
    
    FF10NonRoadDataFrame(df::DataFrame) = new(FF10NonPointDataFrame(df).df)
end

struct FF10OnRoadDataFrame <: EmissionsDataFrame
    df::DataFrame
    
    FF10OnRoadDataFrame(df::DataFrame) = new(FF10NonPointDataFrame(df).df)
end

# https://www.cmascenter.org/smoke/documentation/4.8.1/html/ch08s02s08.html#sect_input_ptinv_ff10
struct FF10PointDataFrame <: EmissionsDataFrame
    df::DataFrame
    
    FF10PointDataFrame(df::DataFrame) = begin
        if size(df, 2) != 77
            throw(DimensionMismatch("FF10 point file should have 77 fields but instead has $(size(df,2))"))
        end
        
        rename!(df, [
            "COUNTRY","FIPS","TRIBAL_CODE","FACILITY_ID","UNIT_ID","REL_POINT_ID","PROCESS_ID","AGY_FACILITY_ID","AGY_UNIT_ID",                
            "AGY_REL_POINT_ID","AGY_PROCESS_ID","SCC","POLID","ANN_VALUE","ANN_PCT_RED","FACILITY_NAME","ERPTYPE","STKHGT",
            "STKDIAM","STKTEMP","STKFLOW","STKVEL","NAICS","LONGITUDE","LATITUDE","LL_DATUM","HORIZ_COLL_MTHD","DESIGN_CAPACITY",
            "DESIGN_CAPACITY_UNITS","REG_CODES","FAC_SOURCE_TYPE","UNIT_TYPE_CODE","CONTROL_IDS","CONTROL_MEASURES",
            "CURRENT_COST","CUMULATIVE_COST","PROJECTION_FACTOR","SUBMITTER_FAC_ID","CALC_METHOD","DATA_SET_ID","FACIL_CATEGORY_CODE",
            "ORIS_FACILITY_CODE","ORIS_BOILER_ID","IPM_YN","CALC_YEAR","DATE_UPDATED","FUG_HEIGHT","FUG_WIDTH_XDIM","FUG_LENGTH_YDIM",
            "FUG_ANGLE","ZIPCODE","ANNUAL_AVG_HOURS_PER_YEAR","JAN_VALUE", "FEB_VALUE", "MAR_VALUE",
            "APR_VALUE", "MAY_VALUE", "JUN_VALUE", "JUL_VALUE", "AUG_VALUE", "SEP_VALUE", "OCT_VALUE", "NOV_VALUE", "DEC_VALUE",
            "JAN_PCTRED", "FEB_PCTRED", "MAR_PCTRED", "APR_PCTRED", "MAY_PCTRED", "JUN_PCTRED", "JUL_PCTRED", "AUG_PCTRED",
            "SEP_PCTRED", "OCT_PCTRED", "NOV_PCTRED", "DEC_PCTRED", "COMMENT"    
        ])
        
        df.ANN_VALUE = df.ANN_VALUE * tonperyear
        
        df.JAN_VALUE = df.JAN_VALUE * tonpermonth
        df.FEB_VALUE = df.FEB_VALUE * tonpermonth
        df.MAR_VALUE = df.MAR_VALUE * tonpermonth
        df.APR_VALUE = df.APR_VALUE * tonpermonth
        df.MAY_VALUE = df.MAY_VALUE * tonpermonth
        df.JUN_VALUE = df.JUN_VALUE * tonpermonth
        df.JUL_VALUE = df.JUL_VALUE * tonpermonth
        df.AUG_VALUE = df.AUG_VALUE * tonpermonth
        df.SEP_VALUE = df.SEP_VALUE * tonpermonth
        df.OCT_VALUE = df.OCT_VALUE * tonpermonth
        df.NOV_VALUE = df.NOV_VALUE * tonpermonth
        df.DEC_VALUE = df.DEC_VALUE * tonpermonth
        
        df.STKHGT = df.STKHGT * foot
        df.STKDIAM = df.STKDIAM * foot
        df.STKTEMP = kelvin.(df.STKTEMP)
        df.STKFLOW = df.STKFLOW * foot * foot * foot / u"s"
        df.STKVEL = df.STKVEL * foot / u"s"
        
        # Fill in missing parameters.
        flowmissing = ismissing.(df.STKFLOW)
        df.STKFLOW[flowmissing] .= df.STKVEL[flowmissing] .* (df.STKDIAM[flowmissing] .* df.STKDIAM[flowmissing] / 4 * π)
        
        velmissing = ismissing.(df.STKVEL)
        df.STKVEL[velmissing] .= df.STKVEL[velmissing] ./ (df.STKDIAM[velmissing] .* df.STKDIAM[velmissing] / 4 * π)
        
        return new(df)
    end
end

### Configuration

Next, we need to specify which files we are going to load and how we are going to load them.
There are several different file formats, and we need to specify what format each of the files is in.
For now, we are focusing on the files that contain annual emissions (rather than daily or hourly emissions
or activity data), so the files that do not contain annual emissions are commented out for now.

In [48]:
readpoint(f) = FF10PointDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readnonpoint(f) = FF10NonPointDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readnonroad(f) = FF10NonRoadDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readonroad(f) = FF10OnRoadDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))

emisdirs = (
    "." => (
        "airports_SmokeFlatFile_POINT_20200412.csv" => readpoint,
        "egucems_2017NEI_POINT_20200412.csv" => readpoint,
        "egunoncems_2017NEI_POINT_20200412.csv" => readpoint,
        "nonegu_SmokeFlatFile_POINT_20200412.csv" => readpoint,
        "oilgas_SmokeFlatFile_POINT_20200412.csv" => readpoint,
    ),
    "2017gb_17j/inputs/afdust" => (
        "afdust_2017NEI_NONPOINT_20200415_27apr2020_nf_v5.csv" => readnonpoint,
    ),
    "2017gb_17j/inputs/ag" => (
        "ag_fertilizer_2017NEI_NONPOINT_20200415_15apr2020_v0.csv" => readnonpoint,
        "ag_livestock_2017NEI_NONPOINT_20200415_17apr2020_nf_v2.csv" => readnonpoint,
    ),
    #2017gb_17j/inputs/cem:
    #HOUR_UNIT_2016_12_31dec_2017.txt
    #HOUR_UNIT_2017_01_1-30jan_2017.txt
    #HOUR_UNIT_2017_01_31jan_2017.txt
    #HOUR_UNIT_2017_02_1-27feb_2017.txt
    #HOUR_UNIT_2017_02_28feb_2017.txt
    #HOUR_UNIT_2017_03_1-30mar_2017.txt
    #HOUR_UNIT_2017_03_31mar_2017.txt
    #HOUR_UNIT_2017_04_1-29apr_2017.txt
    #HOUR_UNIT_2017_04_30apr_2017.txt
    #HOUR_UNIT_2017_05_1-30may_2017.txt
    #HOUR_UNIT_2017_05_31may_2017.txt
    #HOUR_UNIT_2017_06_1-29jun_2017.txt
    #HOUR_UNIT_2017_06_30jun_2017.txt
    #HOUR_UNIT_2017_07_1-30jul_2017.txt
    #HOUR_UNIT_2017_07_31jul_2017.txt
    #HOUR_UNIT_2017_08_1-30aug_2017.txt
    #HOUR_UNIT_2017_08_31aug_2017.txt
    #HOUR_UNIT_2017_09_1-29sep_2017.txt
    #HOUR_UNIT_2017_09_30sep_2017.txt
    #HOUR_UNIT_2017_10_1-30oct_2017.txt
    #HOUR_UNIT_2017_10_31oct_2017.txt
    #HOUR_UNIT_2017_11_1-29nov_2017.txt
    #HOUR_UNIT_2017_11_30nov_2017.txt
    #HOUR_UNIT_2017_12_1-30dec_2017.txt
    #HOUR_UNIT_2017_12_31dec_2017.txt
    "2017gb_17j/inputs/cmv_c1c2" => (
        "canada_c1c2_point_2015_aisremoved_07aug2019_v0.csv" => readpoint,
        #cmv_c1c2_2017adjust_20200422_12US1_2017_10_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_10_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_11_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_11_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_12_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_12_CA_nexthour.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_12_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_12_US_nexthour.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_1_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_1_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_2_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_2_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_3_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_3_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_4_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_4_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_5_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_5_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_6_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_6_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_7_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_7_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_8_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_8_US_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_9_CA_hourly.csv
        #cmv_c1c2_2017adjust_20200422_12US1_2017_9_US_hourly.csv
        "cmv_c1c2_2017adjust_20200422_12US1_2017_CA_annual_22apr2020_v0.csv" => readpoint,
        "cmv_c1c2_2017adjust_20200422_12US1_2017_US_annual_22apr2020_v0.csv" => readpoint,
    ),
    "2017gb_17j/inputs/cmv_c3" => (
        "Mexico_2014v1_CMV_point_aisremoved_07aug2019_v0.csv" => readpoint,
        "canada_c3_point_2015_aisremoved_05aug2019_v0.csv" => readpoint,
        #cmv_c3_2017adjust_20200422_12US1_2017_10_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_10_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_11_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_11_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_12_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_12_CA_nexthour.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_12_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_12_US_nexthour.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_1_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_1_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_2_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_2_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_3_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_3_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_4_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_4_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_5_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_5_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_6_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_6_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_7_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_7_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_8_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_8_US_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_9_CA_hourly.csv
        #cmv_c3_2017adjust_20200422_12US1_2017_9_US_hourly.csv
        "cmv_c3_2017adjust_20200422_12US1_2017_CA_annual_25apr2020_v0.csv" => readpoint,
        "cmv_c3_2017adjust_20200422_12US1_2017_US_annual_25apr2020_v0.csv" => readpoint,
        "ptinv_2016ff_proj_from_eca_imo_nonUS_nonCANADA_caps_vochaps_2011_aisremoved_05aug2019_v0.csv" => readpoint,
    ),
    "2017gb_17j/inputs/nonpt" => (
        "2017NEI_NONPOINT_20200415_15apr2020_nf_v1.csv" => readnonpoint,
    ),
    "2017gb_17j/inputs/nonroad" => (
        "2017nei_nonroad_california_15apr2020_nf_v4.csv" => readnonroad,
        "2017nei_nonroad_from_MOVES2014b_aggSCC_15apr2020_nf_v6.csv" => readnonroad,
    ),
    "2017gb_17j/inputs/np_oilgas" => (
        "np_oilgas_2017NEI_NONPOINT_20200501_04may2020_nf_v1.csv" => readnonpoint,
    ),
    # 2017gb_17j/inputs/onroad:
    # HOTELING_2017NEI_27mar2020_nf_v4.csv
    # SPEED_2017NEI_from_CDBs_27mar2020_nf_v5.csv
    # VMT_2017NEI_final_from_CDBs_month_redist_27mar2020_nf_v4.csv
    # VPOP_2017NEI_final_from_CDBs_27mar2020_nf_v6.csv

    # 2017gb_17j/inputs/onroad_ca_adj:
    # HOTELING_2017NEI_27mar2020_nf_v5.csv
    # SPEED_2017NEI_from_CDBs_27mar2020_nf_v6.csv
    # VMT_2017NEI_final_from_CDBs_month_redist_27mar2020_nf_v5.csv
    # VPOP_2017NEI_final_from_CDBs_27mar2020_nf_v7.csv

    # 2017gb_17j/inputs/onroad_can:
    # canada_2017interp_transport_onroad_monthly_svn70_11feb2020_v0.csv
    # canada_2017interp_transport_onroad_refueling_svn70_11feb2020_v0.csv

    "2017gb_17j/inputs/onroad_mex" => (
        #"Mexico_2017_onroad_MOVES_aggSCC_29aug2017_v0.csv" => readonroad, # File is empty!
    ),
    "2017gb_17j/inputs/ptagfire" => (
        #ptday_agburn_2017_MYR_ff10_22apr2020_v0
        #ptday_agburn_2017_MYR_prevdec_22apr2020_v0
        #"ptinv_agburn_2017_MYR_ff10_22apr2020_v0.csv" => readpoint,
    ),
    "2017gb_17j/inputs/ptfire" => (
        # ptday_ptfire_2017NEI_20200206_CONUS_CAPs_07apr2020_nf_v1
        # ptday_ptfire_2017NEI_20200206_CONUS_CAPs_prevdec_07apr2020_nf_v1
        # ptday_ptfire_2017NEI_20200407_CONUS_HAPs_07apr2020_v0
        # ptday_ptfire_2017NEI_20200407_CONUS_HAPs_prevdec_07apr2020_v0
        # "ptinv_ptfire_2017NEI_20200206_CONUS_CAPs_07apr2020_nf_v1.csv" => readpoint,
        # "ptinv_ptfire_2017NEI_20200407_CONUS_HAPs_07apr2020_v0.csv" => readpoint,
    ),
    "2017gb_17j/inputs/ptfire_othna" => (
        # ptday_finn_CA_finn_2017_ff10_20200420_20apr2020_v0
        # ptday_finn_CA_finn_2017_ff10_20200420_prevdec_21apr2020_v0
        # ptday_finn_CB_finn_2017_ff10_20200420_20apr2020_v0
        # ptday_finn_CB_finn_2017_ff10_20200420_prevdec_21apr2020_v0
        # ptday_finn_CT_finn_2017_ff10_20200420_20apr2020_v0
        # ptday_finn_CT_finn_2017_ff10_20200420_prevdec_21apr2020_v0
        # ptday_finn_DR_finn_2017_ff10_20200420_20apr2020_v0
        # ptday_finn_DR_finn_2017_ff10_20200420_prevdec_21apr2020_v0
        # ptday_finn_HT_finn_2017_ff10_20200420_20apr2020_v0
        # ptday_finn_HT_finn_2017_ff10_20200420_prevdec_21apr2020_v0
        # ptday_finn_JA_finn_2017_ff10_20200420_20apr2020_v0
        # ptday_finn_JA_finn_2017_ff10_20200420_prevdec_21apr2020_v0
        # ptday_finn_MX_finn_2017_ff10_20200420_20apr2020_v0
        # ptday_finn_MX_finn_2017_ff10_20200420_prevdec_21apr2020_v0
        # "ptinv_finn_CA_finn_2017_ff10_20200420_21apr2020_v0.csv" => readpoint,
        # "ptinv_finn_CB_finn_2017_ff10_20200420_21apr2020_v0.csv" => readpoint,
        # "ptinv_finn_CT_finn_2017_ff10_20200420_21apr2020_v0.csv" => readpoint,
        # "ptinv_finn_DR_finn_2017_ff10_20200420_21apr2020_v0.csv" => readpoint,
        # "ptinv_finn_HT_finn_2017_ff10_20200420_21apr2020_v0.csv" => readpoint,
        # "ptinv_finn_JA_finn_2017_ff10_20200420_21apr2020_v0.csv" => readpoint,
        # "ptinv_finn_MX_finn_2017_ff10_20200420_21apr2020_v0.csv" => readpoint,
    ),
    "2017gb_17j/inputs/rail" => (
        "rail_2017NEI_NONPOINT_20200415_15apr2020_v0.csv" => readnonpoint,
    ),
    "2017gb_17j/inputs/rwc" => (
        "rwc_2017NEI_NONPOINT_20200415_15apr2020_v0.csv" => readnonpoint,
    ),
);

### Load emissions

Now we iterate through each of the files above and read them into dataframes. Each file contains a lot of data, but we're only interested in the pollutant name, the source classification code (SCC) the describes the type of emitter, and the annual emissions rate. So for now we only save those three variables

In [ ]:
dfs = []
for (emisdir, data) ∈ emisdirs
    for (file, readfunc) ∈ data
        path = joinpath(dir, emisdir, file)
        df = readfunc(path)
        push!(dfs, df.df[!, [:POLID, :SCC, :ANN_VALUE]])
    end
end

### Results

Next, let's combine all the files together into one big dataframe.
We can see that altogether, there are about 34 million emission records 
in this dataset that we're processing so far.

In [50]:
emis = vcat(dfs...)

,POLID,SCC,ANN_VALUE
,String15,Int64,Quantity…
1,CO2,39999999,0.768383 kg s⁻¹
2,N2O,39999999,1.45865e-6 kg s⁻¹
3,CH4,39999999,1.45865e-5 kg s⁻¹
4,N2O,39999999,4.34424e-6 kg s⁻¹
5,CH4,39999999,4.2174e-5 kg s⁻¹
6,CO2,39999999,2.20894 kg s⁻¹
7,91576,2275050011,0.0 kg s⁻¹
8,91203,2275050011,6.32378e-9 kg s⁻¹
9,86737,2275050011,1.01201e-10 kg s⁻¹


A lot of the records above are duplicates as far as pollutant and SCC are concerned. If we combine all those, we can see that there are about 124 thousand unique combintions of pollutant and SCC.

In [51]:
emis_grouped = sort(combine(groupby(emis, [:POLID, :SCC]), :ANN_VALUE=>sum=>:ANN_VALUE), :ANN_VALUE, rev=true)

,POLID,SCC,ANN_VALUE
,String15,Int64,Quantity…
1,CO2,39999999,81525.0 kg s⁻¹
2,CO2,2270002022,1973.72 kg s⁻¹
3,CO2,2275020000,1494.62 kg s⁻¹
4,CO2,2270005022,1466.93 kg s⁻¹
5,BSFC,2270002022,567.775 kg s⁻¹
6,BSFC,2270005022,449.858 kg s⁻¹
7,CO2,2267003022,423.005 kg s⁻¹
8,CO2,2280002203,340.402 kg s⁻¹
9,CO2,2265004022,296.028 kg s⁻¹


There are 340 unique pollutants:

In [53]:
unique(emis.POLID)

340-element Vector{String15}:
 "CO2"
 "N2O"
 "CH4"
 "91576"
 "91203"
 "86737"
 "85018"
 "83329"
 "75070"
 "7439921"
 "71432"
 "67561"
 "56553"
 ⋮
 "NONHAPTOG1001"
 "NONHAPTOG8775"
 "NONHAPTOG95331"
 "NONHAPTOG95332"
 "NONHAPTOG95333"
 "NONHAPTOG95335A"
 "PM25_8996"
 "PM25_91106"
 "DPM25TOTAL"
 "DPM25_8996"
 "DPM25_91106"
 "BSFC"

...and 5,400 unique SCC codes:

In [54]:
unique(emis.SCC)

5368-element Vector{Int64}:
   39999999
 2275050011
 2275050012
 2275060012
 2275060011
 2275001000
 2275020000
 2275070000
 2265008005
 2270008005
   20100201
   20100101
   10100601
          ⋮
 2104008700
 2104008310
 2104008100
 2104008400
 2104008330
 2104008610
 2104009000
 2104008230
 2104008300
 2104008530
 2104008620
 2104008630

Of all the pollutants in the dataset, CO<sub>2</sub> has the most overall emissions.

In [58]:
sort(combine(groupby(emis[!, [:POLID, :ANN_VALUE]], :POLID), :ANN_VALUE => sum => :total), :total, rev=true)

,POLID,total
,String15,Quantity…
1,CO2,93836.5 kg s⁻¹
2,BSFC,1908.79 kg s⁻¹
3,CO,544.156 kg s⁻¹
4,PM10-PRI,524.509 kg s⁻¹
5,PM10-FIL,511.445 kg s⁻¹
6,NOX,233.974 kg s⁻¹
7,VOC,223.74 kg s⁻¹
8,CH4,137.63 kg s⁻¹
9,NH3,105.56 kg s⁻¹


As another quick exploration, we can see that the SCC code 2280003200 is associated with the largest amount of NO<sub>x</sub> emissions. As shown in the EPA's [SCC explorer](https://sor-scc-api.epa.gov/sccwebservices/sccsearch/), this code corresponds to commercial marine vessels, while underway, burning residual fuel.

In [61]:
sort(filter(:POLID => ==("NOX"), emis_grouped), :ANN_VALUE, rev=true)

,POLID,SCC,ANN_VALUE
,String15,Int64,Quantity…
1,NOX,2280003200,35.9467 kg s⁻¹
2,NOX,2285002006,14.064 kg s⁻¹
3,NOX,2270005022,10.037 kg s⁻¹
4,NOX,2270002022,9.1068 kg s⁻¹
5,NOX,2280002203,8.75787 kg s⁻¹
6,NOX,2280003203,6.66219 kg s⁻¹
7,NOX,10100202,6.19822 kg s⁻¹
8,NOX,10100222,5.96167 kg s⁻¹
9,NOX,2104006000,5.90911 kg s⁻¹


### Next steps

Our next steps will be to perform some spatial and temporal processing on these emissions. To do this, we will pull in the EPA's spatial surrogate shapefiles to allocate county-level emissions to locations within each county, and we will leverage the daily and hourly emission files above plus temporal surrogate information to allocate the emissions to times during the year. Finally, we will leverage emissions factor and meteorology information to convert the mobile and fire activity data above into emissions.

Stay tuned!